## Optimizer: Finding best hyper-parameters

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install "numpy<1.24.0"

In [3]:
pip install transformers datasets scikit-optimize

  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached scikit_optimize-0.9.0-py2.py3-none-any.whl (100 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached pyaml-23.12.0-py3-none-any.whl (23 kB)


In [4]:
pip install accelerate -U

In [5]:
pip install transformers[torch]

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path

from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from skopt import gp_minimize

In [7]:
#Define the space of hyperparameters to search
search_space = [
  Real(1e-5, 5e-5, name='learning_rate'),
  Real(0.01, 0.2,  name='weight_decay'),
  Integer(3, 5, name='num_train_epochs'),
  Integer(8, 32, name='per_device_train_batch_size', prior='log-uniform'),
  Integer(8, 64, name='per_device_eval_batch_size', prior='log-uniform'),
]


@use_named_args(search_space)
def objective(learning_rate, num_train_epochs, per_device_train_batch_size, per_device_eval_batch_size, weight_decay):
    __file__ = "/content/drive/MyDrive/FinalProject/Models/RoBERTa/RoBERTa _optimizer.ipynb"
    file_name = "dataset_balanced_4000"
    ext ="xlsx"
    path_type = "Balanced"

    current_file_path = Path(__file__).parent
    path_to_project = current_file_path.parents[1]

    df = pd.read_excel(f"{path_to_project}/Data/Datasets/{path_type}/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/RoBERTa/Output/{path_type}/{file_name}"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    # Select the text and label columns
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)
    X = df['review'].values
    y = df['label'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


    # Encode the labels to a numeric format
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Initialize the tokenizer for RoBERTa
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    # Tokenization function
    def tokenize_function(texts):
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)


    # Tokenize the data
    train_encodings = tokenize_function(X_train.tolist())
    val_encodings = tokenize_function(X_test.tolist())

    # Create dataset objects
    train_dataset = ReviewDataset(train_encodings, y_train_encoded)
    val_dataset = ReviewDataset(val_encodings, y_test_encoded)

    # Initialize the model for each fold
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))

    # Define training arguments for each fold, adjust hyperparameters as needed
    training_args = TrainingArguments(
        output_dir=f"{dump_dir}/res",
        num_train_epochs= int(num_train_epochs),
        per_device_train_batch_size= int(per_device_train_batch_size),
        per_device_eval_batch_size= int(per_device_eval_batch_size),
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir=f"{dump_dir}/logs",
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        max_grad_norm=1.0,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_strategy="epoch",
        save_total_limit=2,
        lr_scheduler_type='linear'
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(-1), p.label_ids)}
        )

    # Train
    trainer.train()
    # Evaluate
    results = trainer.evaluate()
    neg_accuracy = -results['eval_accuracy']

    print(results['eval_accuracy'])
    print(neg_accuracy)

    shutil.rmtree(dump_dir)
    return neg_accuracy

# Custom dataset class
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

result = gp_minimize(objective, search_space, n_calls=20, random_state=0)

print("Best parameters:")
print(result.x)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.023100,0.784544,0.678750
2,0.700700,0.713726,0.746250
3,0.413000,0.827384,0.771250
4,0.222900,0.994435,0.783750


0.78375
-0.78375


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.008200,0.811050,0.658750
2,0.723700,0.671443,0.758750
3,0.415900,0.805371,0.782500


0.7825
-0.7825
Best parameters:
[4.478758514361407e-05, 0.17277768585588338, 5, 10, 8]
